先来解释一下句法分析。句法分析分为句法结构分析和依存关系分析。  

句法结构分析也就是短语结构分析，比如提取出句子中的名次短语、动词短语等，最关键的是人可以通过经验来判断的短语结构，那么怎么由机器来判断呢？

## 1. 句法分析树
样子如下：  
&emsp;&emsp;       -吃（v）-  
&emsp;    | &emsp;&emsp;&emsp;&emsp;   |  
&emsp;    我（rr）&emsp;     肉（n）   
<br/> 
### 1.1 句法结构分析基本方法
分为基于规则的分析方法和基于统计的分析方法。基于规则的方法存在很多局限性，所以我们采取基于统计的方法，目前最成功的是基于概率上下文无关文法(PCFG)。基于PCFG分析需要有如下几个要素：终结符集合、非终结符集合、规则集。  

相对于先叙述理论再举实例的传统讲解方法，我更倾向于先给你展示一个简单的例子，先感受一下计算过程，然后再叙述理论，这样会更有趣。  

例子是这样的：我们的终结符集合是：∑={我, 吃, 肉,……}，这个集合表示这三个字可以作为句法分析树的叶子节点，当然这个集合里还有很多很多的词  

我们的非终结符集合是：N={S, VP, ……}，这个集合表示树的非页子节点，也就是连接多个节点表达某种关系的节点，这个集合里也是有很多元素  

我们的规则集：R={  
NN->我&emsp;     0.5  
Vt->吃&emsp;      1.0  
NN->肉&emsp;    0.5  
VP->Vt NN&emsp;     1.0  
S->NN VP&emsp;  1.0  
……  
}    

这里的句法规则符号可以参考词性标注，后面一列是模型训练出来的概率值，也就是在一个固定句法规则中NN的位置是“我”的概率是0.5，NN推出“肉”的概率是0.5，0.5+0.5=1，也就是左部相同的概率和一定是1。不知道你是否理解了这个规则的内涵   
<br/> 
再换一种方法解释一下，有一种句法规则是：  
S——|    
|&emsp; |  
NN &emsp;VP  
&emsp;&emsp;&emsp;|———|  
&emsp;&emsp;&emsp;Vt &emsp;NN  

其中NN的位置可能是“我”，也可能是“肉”，是“我”的概率是0.5，是“肉”的概率是0.5，两个概率和必为1。其中Vt的位置一定是“吃”，也就是概率是1.0……。这样一说是不是就理解了？  

规则集里实际上还有很多规则，只是列举出会用到的几个  
<br/> 
那么如何根据以上的几个要素来生成句法分析树呢？  
（1）“我”  
词性是NN，推导概率是0.5，树的路径是“我”  
（2）“吃”  
词性是Vt，推导概率是1.0，树的路径是“吃”  
（3）“肉”  
词性是NN，概率是0.5，和Vt组合符合VP规则，推导概率是0.5*1.0*1.0=0.5，树的路径是“吃肉”
NN和VP组合符合S规则，推导概率是0.5*0.5*1.0=0.25，树的路径是“我吃肉” 

所以最终的树结构是：  

S——|  
|&emsp;|  
NN  VP  
我&emsp;|——|  
&emsp; Vt &emsp; NN  
&emsp;&emsp;吃 &emsp;肉   

上面的例子是比较简单的，实际的句子会更复杂，但是都是通过这样的动态规划算法完成的  

提到动态规划算法，就少不了“选择”的过程，一句话的句法结构树可能有多种，我们只选择概率最大的那一种作为句子的最佳结构，这也是“基于概率”上下文无关文法的名字起源。  

上面的计算过程总结起来就是：设W={ω1ω2ω3……}表示一个句子，其中的ω表示一个词(word)，利用动态规划算法计算非终结符A推导出W中子串ωiωi+1ωi+2……ωj的概率，假设概率αij(A)，那么有如下递归公式：    
αij(A)=P(A->ωi)  
αij(A)=∑∑P(A->BC)αik(B)α(k+1)j(C)    
以上两个式子好好理解一下其实就是上面“我吃肉”的计算过程  
以上过程理解了之后你一定会问，这里面最关键的的非终结符、终结符以及规则集是怎么得来的，概率又是怎么确定的？下面我们就来说明

## 2. 句法规则提取方法与PCFG的概率参数估计
首先我们需要大量的树库，也就是训练数据。然后我们把树库中的句法规则提取出来生成我们想要的结构形式，并进行合并、归纳等处理，最终得到上面∑、N、R的样子。其中的概率参数计算方法是这样的：  

先给定参数为一个随机初始值，然后采用EM迭代算法，不断训练数据，并计算每条规则使用次数作为最大似然计算得到概率的估值，这样不断迭代更新概率，最终得出的概率可以认为是符合最大似然估计的精确值。

句法分析树生成算法是基于统计学习的原理，根据大量标注的语料库（树库），通过机器学习算法得出非终结符、终结符、规则集及其概率参数，然后利用动态规划算法生成每一句话的句法分析树，在句法分析树生成过程中如果遇到多种树结构，选择概率最大的那一种作为最佳句子结构

## 3. 词义消歧
词义消歧是句子和篇章语义理解的基础，是必须解决的问题。任何一种语言都有大量具有多种含义的词汇，中文的“日”，英文的“bank”，法语的“prendre”……。  

词义消歧可以通过机器学习的方法来解决。谈到机器学习就会分成有监督和无监督的机器学习。词义消歧有监督的机器学习方法也就是分类算法，即判断词义所属的分类。词义消歧无监督的机器学习方法也就是聚类算法，把词义聚成多类，每一类是一种含义。

## 4. 有监督的词义消歧方法
<br/> 
### 4.1 基于互信息的词义消歧方法
这个方法的名字不好理解，但是原理却非常简单：用两种语言对照着看，比如：中文“打人”对应英文“beat a man”，而中文“打酱油”对应英文“buy some sauce”。这样就知道当上下文语境里有“人”的时候“打”的含义是beat，当上下文语境里有“酱油”的时候“打”的含义是buy。按照这种思路，基于大量中英文对照的语料库训练出来的模型就可以用来做词义消歧了，这种方法就叫做基于“互信息”的词义消歧方法。讲到“互信息”还要说一下它的起源，它来源于信息论，表达的是一个随机变量中包含另一个随机变量的信息量(也就是英文信息中包含中文信息的信息量)，$假设两个随机变量X、Y的概率分别是p(x), p(y)，它们的联合分布概率是p(x,y)，那么互信息计算公式是：$  
$$ I(X; Y)=\sum\sum P(x,y)log(\frac{P(x,y)}{P(x)P(y)}) $$
以上公式是怎么推导出来的呢？比较简单，“互信息”可以理解为一个随机变量由于已知另一个随机变量而减少的不确定性(也就是理解中文时由于已知了英文的含义而让中文理解更确定了)，因为“不确定性”就是熵所表达的含义，所以：  
$$I(X; Y)=H(X)-H(X|Y)$$
等式后面经过不断推导就可以得出上面的公式。  

那么我们在对语料不断迭代训练过程中I(X; Y)是不断减小的，算法终止的条件就是I(X; Y)不再减小。  

基于互信息的词义消歧方法自然对机器翻译系统的效果是最好的，但它的缺点是：双语语料有限，多种语言能识别出歧义的情况也是有限的(比如中英文同一个词都有歧义就不行了)。  

### 4.2 基于贝叶斯分类器的消歧方法
提到贝叶斯那么一定少不了条件概率，这里的条件指的就是上下文语境这个条件，任何多义词的含义都是跟上下文语境相关的。假设语境(context)记作c，语义(semantic)记作s，多义词(word)记作w，那么我要计算的就是多义词w在语境c下具有语义s的概率，即：$P(s|c)$  
<br/> 
那么根据贝叶斯公式：$$P(s|c)=\frac{P(c|s)P(s)}{P(c)}$$   
<br/> 
$我要计算的就是p(s|c)中s取某一个语义的最大概率，因为p(c)是既定的，所以只考虑分子的最大值：$ $s的估计=max(P(c|s)P(s))$  
<br/> 
因为语境c在自然语言处理中必须通过词来表达，也就是有多个v(词)组成，  
也就是计算：$max(P(s)\prod(v|s))$  
<br/> 
下面就是训练过程了：
$P(s)代表的是多义词w的某个语义s的概率，可以统计大量语料通过最大似然估计求得$：  
$$P(s)=\frac{N(s)}{W(w)}$$

$P(v|s)表达的是多义词w的某个语义s的条件下出现词v的概率，可以统计大量语料通过最大似然估计求得：$ $$P(v|s)=\frac{N(v,s}{N(s)})$$  
$训练出P(s)和P(v|s)之后我们对一个多义词w消歧的过程就是计算$ $P(c|s)P(s)$

## 5. 无监督的词义消歧方法
完全无监督的词义消歧是不可能的，因为没有标注是无法定义是什么词义的，但是可以通过无监督的方法来做词义辨识。无监督的词义辨识其实也是一种贝叶斯分类器，和上面讲到的贝叶斯分类器消歧方法不同在于：这里的参数估计不是基于有标注的训练预料，而是先随机初始化参数p(v|s)，然后根据EM算法重新估计这个概率值，也就是对w的每一个上下文c计算p(c|s)，这样可以得到真实数据的似然值，回过来再重新估计p(v|s)，重新计算似然值，这样不断迭代不断更新模型参数，最终得到分类模型，可以对词进行分类，那么有歧义的词在不同语境中会被分到不同的类别里。

仔细思考一下这种方法，其实是基于单语言的上下文向量的，那么我们进一步思考下一话题，如果一个新的语境没有训练模型中一样的向量怎么来识别语义？

这里就涉及到向量相似性的概念了，我们可以通过计算两个向量之间夹角余弦值来比较相似性，即：  
$$cos(a,b)=\frac{\sum{ab}}{\sqrt{\sum{a^{2}}\sum{b^{2}}}}$$